In [1]:
import os
# import mapclassify
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display
from mpl_toolkits.axes_grid1 import make_axes_locatable
from datetime import datetime

import geopandas as geo
import pysal as ps


%matplotlib inline

ImportError: DLL load failed while importing ogrext: %1 is not a valid Win32 application.

#### Import Dataset

In [ ]:
root = 'C:/Users/Aaron/PycharmProjects/ECE-225'
data_path = Path(root + '/data/output.csv')
df = pd.read_csv(data_path)
print('Rate Max: ' + str(df.loc[:,'Rate'].max()))
print('Rate Min.: ' + str(df.loc[:,'Rate'].min()))
print('Rate Mean: ' + str(df.loc[:,'Rate'].mean()))
print('Rate Std.: ' + str(df.loc[:,'Rate'].std()))   

# df.loc[:,'Rate'] = ((df.loc[:,'Rate']-df.loc[:,'Rate'].mean())/df.loc[:,'Rate'].std())

In [ ]:
df['Date'] = df.apply(lambda x: x.Month+' '+str(x.Year), axis=1)
df.Date = pd.to_datetime(df.Date)

In [ ]:
df.drop(columns=['Year', 'Month'], inplace=True)

In [ ]:
df.head()

#### Import .shp file data for counties

In [ ]:
shp_counties = Path(root + "/data/UScounties/uscounties.shp")

c_map = geo.read_file(shp_counties)
c_map = c_map[c_map.STATE_NAME != 'Alaska']
c_map = c_map[c_map.STATE_NAME != 'Hawaii']
c_map.rename(columns={'NAME':'County', 'STATE_NAME':'State'}, inplace=True)
c_map.drop(columns=['STATE_FIPS', 'CNTY_FIPS', 'FIPS'], inplace=True)

df.convert_dtypes()
c_map.convert_dtypes()


In [ ]:
c_map.head()

In [ ]:
# Clean Data
df['County'] = df.County.str.replace(' County', '')
print(df.head())
print(np.sum(df[['State', 'County']].isin({'County':['Newton'], 'State':['Mississippi']})))

In [ ]:
print(np.sum(c_map[['State', 'County']].isin({'County':['Newton'], 'State':['Mississippi']})))

In [ ]:
print('Unique Counties in Data: '+ str(np.array(df['County'].unique()).shape))
print('Uniqeu counties in SHP: '+ str(np.array(c_map['County'].unique()).shape))

### Plot with Slider

In [ ]:
print(df.head())

In [ ]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior
        Sources: https://automating-gis-processes.github.io/2016/Lesson5-interactive-map-bokeh.html
                 https://stackoverflow.com/questions/55659835/trying-to-separate-polygon-data-into-x-and-y-coordinates-but-get-error-multip
   """
    # Parse the geometries and grab the coordinate
    geometry = row[geom]
    #print(geometry.type)

    if geometry.type=='Polygon':
        if coord_type == 'x':
            # Get the x coordinates of the exterior
            # Interior is more complex: xxx.interiors[0].coords.xy[0]
            return list( geometry.exterior.coords.xy[0] )
        elif coord_type == 'y':
            # Get the y coordinates of the exterior
            return list( geometry.exterior.coords.xy[1] )

    if geometry.type in ['Point', 'LineString']:
        if coord_type == 'x':
            return list( geometry.xy[0] )
        elif coord_type == 'y':
            return list( geometry.xy[1] )

    if geometry.type=='MultiLineString':
        all_xy = []
        for ea in geometry:
            if coord_type == 'x':
                all_xy.append(list( ea.xy[0] ))
            elif coord_type == 'y':
                all_xy.append(list( ea.xy[1] ))
        return all_xy

    if geometry.type=='MultiPolygon':
        all_xy = []
        for ea in geometry:
            if coord_type == 'x':
                all_xy.append(list( ea.exterior.coords.xy[0] ))
            elif coord_type == 'y':
                all_xy.append(list( ea.exterior.coords.xy[1] ))
        return all_xy

    else:
        # Finally, return empty list for unknown geometries
        return []

In [ ]:
# Bokeh Plot Imports

from bokeh.palettes import RdYlBu11 as palette
from bokeh.plotting import figure, output_file, show, save
from bokeh.transform import linear_cmap
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper, ColorBar
from bokeh.io import output_notebook, push_notebook
output_notebook()

In [ ]:
# Bokeh map
grid = c_map.copy()
lines = c_map.copy()

In [ ]:
# Process patches and lines for map

grid['x'] = grid.apply(func=getPolyCoords, geom = 'geometry', coord_type='x', axis=1)
grid['y'] = grid.apply(func=getPolyCoords, geom='geometry', coord_type='y', axis=1)

lines['x'] = lines.apply(func=getPolyCoords, geom = 'geometry', coord_type='x', axis=1)
lines['y'] = lines.apply(func=getPolyCoords, geom='geometry', coord_type='y', axis=1)

#  Merge data with coordinates
dg = geo.geodataframe.GeoDataFrame(grid.merge(df, on=['County', 'State']), geometry='geometry')

g_df = dg[dg.Date==datetime(2011, 1, 1)].drop('geometry', axis=1).copy()
m_df = lines.drop('geometry', axis=1).copy()

msource = ColumnDataSource(m_df)
gsource = ColumnDataSource(g_df)

In [ ]:
print(g_df.head())

In [ ]:
# Initialize our figure
p = figure(title="Unemloyment Rates 2016 January", plot_width=1300, plot_height=700, output_backend="webgl")

# Create the color mapper
color_mapper = linear_cmap(field_name='Rate', palette=palette ,low=min(g_df.Rate) ,high=max(g_df.Rate))

# Add Color Bar
color_bar = ColorBar(color_mapper=color_mapper['transform'], width=8,  location=(0,0))
p.add_layout(color_bar, 'right')

# Add Hover Tool
hov = HoverTool()
hov.tooltips = [('County', '@County'), ('State', '@State'), ('Rate', '@Rate')]
p.add_tools(hov)


# Plot grid
r = p.patches('x', 'y', source=gsource,
         color=color_mapper,
         fill_alpha=1.0, line_color="blue", line_width=0.001)

# Add boundaries to figure
# p.multi_line('x', 'y', source=msource, color="red", line_width=2)

# # Save the figure
# outfp = output_file(r"C:/Users/Aaron/PycharmProjects/ECE-225/unemloyment_map.html")
# save(obj=p, title='unemloyment_map', filename=outfp)

In [ ]:
def update_bokeh(y, m):
    r.data_source.data = dg[dg.Date==datetime(y, m, 1)].drop('geometry', axis=1).copy()
    p.title.text = 'Unemployment Rate %d-%02d'%(y, m)
    push_notebook()

In [ ]:
ye = widgets.IntSlider(min=np.min(df.Date.dt.year), max=np.max(df.Date.dt.year), description="Year", continuous_update=False)
mon = widgets.IntSlider(min=1, max=12, description="Month")
show(p, notebook_handle=True)

In [ ]:
widgets.interact(update_bokeh, y=ye, m=mon)

In [ ]:
months = ["January", "February", "March","April", "May", "June", "July", "August","September", "October","November" , "December" ]
# dg = geo.geodataframe.GeoDataFrame(c_map.merge(df, on=['County', 'State']), geometry='geometry')

def plotmap(y, m):
    fig, ax = plt.subplots(1, figsize=(20,20))
    div = make_axes_locatable(ax)
    cax = div.append_axes("right", size="5%", pad=0.1)
    c_map.boundary.plot(ax=ax, markersize=0.0001, zorder=1)
#     new = geo.geodataframe.GeoDataFrame(c_map.merge(df[(df.Date.dt.year == y) & (df.Date.dt.month == m)], on=['County', 'State'], how='left'), geometry='geometry')
    dg[dg.Date==datetime(y, m, 1)].plot(column='Rate', ax=ax, legend=True, cax=cax, cmap='inferno', zorder=2)



In [ ]:
# slider = widgets.IntSlider(min=1, max=(np.max(df.Date.dt.year)-np.min(df.Date.dt.year))*12)

year_s = widgets.IntSlider(min=np.min(df.Date.dt.year), max=np.max(df.Date.dt.year), description="Year", continuous_update=False)
mon_s = widgets.IntSlider(min=1, max=12, description="Month")
play = widgets.Play(
    value=1990,
    min=1990,
    max=2016,
    step=1,
    interval=500,
    description="Press play",
    disabled=False
)
widgets.jslink((play, 'value'), (year_s, 'value'))
widgets.interact(plotmap, y=year_s, m=mon_s)
widgets.HBox([play, year_s])

In [ ]:
# from arcgis.gis import GIS
# from arcgis.geometry import Geometry
# sdf = pd.DataFrame.spatial.from_featureclass(shp_counties)
# sdf = sdf[sdf.STATE_NAME != 'Alaska']
# sdf = sdf[sdf.STATE_NAME != 'Hawaii']
# sdf.rename(columns={'NAME':'County', 'STATE_NAME':'State'}, inplace=True)
# sdf.drop(columns=['STATE_FIPS', 'CNTY_FIPS', 'FIPS'], inplace=True)
# sdf.tail()

In [ ]:
# my_gis = GIS()
# m = my_gis.map('United States')

# df_g = sdf.merge(df, on=['County', 'State'])
# # df_g = sdf.merge(df[(df.Date.dt.year == 2016) & (df.Date.dt.month == 1)], on=['County', 'State'], how='left')
# print(df_g.shape)
# print(df_g)
# # df_g.head()

In [ ]:
# df_g[df_g.Date == datetime(2016, 1, 1)]
# df_layer[df_g.Date == datetime(2016, 1, 1)].spatial.plot(map_widget=m, col='Rate')
# m